# Project NLP - Automated Customer reviews

In [31]:
# Libraries
import pandas as pd
import numpy as np

# Import and read the Dataset

In [32]:
# Read the data datasets

# Dataset from Sep-Oct 2018 (5k datapoints)
test_data = pd.read_csv("/Users/tiagovhp/Ironhack/Week_7/Amazon_SentimentAnalysis/Dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")

# Dataset from Feb-Apr 2019 (28332 datapoints)
train_data = pd.read_csv("/Users/tiagovhp/Ironhack/Week_7/Amazon_SentimentAnalysis/Dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# Info about the Dataset
display(test_data.info())
display(train_data.info())

# Visualize the Data
#display(train_data.head())
#display(test_data.head())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   5000 non-null   object 
 1   dateAdded            5000 non-null   object 
 2   dateUpdated          5000 non-null   object 
 3   name                 5000 non-null   object 
 4   asins                5000 non-null   object 
 5   brand                5000 non-null   object 
 6   categories           5000 non-null   object 
 7   primaryCategories    5000 non-null   object 
 8   imageURLs            5000 non-null   object 
 9   keys                 5000 non-null   object 
 10  manufacturer         5000 non-null   object 
 11  manufacturerNumber   5000 non-null   object 
 12  reviews.date         5000 non-null   object 
 13  reviews.dateAdded    1052 non-null   object 
 14  reviews.dateSeen     5000 non-null   object 
 15  reviews.doRecommend  5000 non-null   b

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

None

# Data Preprocessing

In [33]:
# Handle missing values
train_data.isna().sum()
test_data.isna().sum()


# Handle missing values = No recommendation
#train_data['reviews.doRecommend'] = train_data['reviews.doRecommend'].fillna("False")
# Replace boolean with string
#train_data['reviews.doRecommend'] = train_data['reviews.doRecommend'].replace({True: 'Yes', False: 'No'})


id                        0
dateAdded                 0
dateUpdated               0
name                      0
asins                     0
brand                     0
categories                0
primaryCategories         0
imageURLs                 0
keys                      0
manufacturer              0
manufacturerNumber        0
reviews.date              0
reviews.dateAdded      3948
reviews.dateSeen          0
reviews.doRecommend       0
reviews.id             4971
reviews.numHelpful        0
reviews.rating            0
reviews.sourceURLs        0
reviews.text              0
reviews.title            13
reviews.username          1
sourceURLs                0
dtype: int64

In [34]:
# Separate data into training & test
# Keep only relevant columns
## reviews.title, reviews.text, reviews.rating, reviews.doRecommend
## Combine reviews.doRecommend, reviews.title and reviews.text into 1 single column: concatenate with " "

# Train data
train = pd.DataFrame(train_data['reviews.text'], columns = ['reviews.text'])
train['reviews.title'] = train_data['reviews.title']
#train['reviews.doRecommend'] = train_data['reviews.doRecommend']
train['combined.review'] = train['reviews.title'] + ' ' + train['reviews.text']
#train['combined.review'] = train_data['reviews.doRecommend'] + ' ' + train['combined.review']
train = train.drop(columns=["reviews.title","reviews.text"])
train['reviews.rating'] = train_data['reviews.rating']
display(train.head())

# Test data
test = pd.DataFrame(test_data['reviews.text'], columns = ['reviews.text'])
test['reviews.title'] = test_data['reviews.title']
test['combined.review'] = test['reviews.title'] + ' ' + test['reviews.text']
test = test.drop(columns=["reviews.title","reviews.text"])
test['reviews.rating'] = test_data['reviews.rating']
display(test.head())

,combined.review,reviews.rating
0,... 3 of them and one of the item is bad quali...,3
1,... always the less expensive way to go for pr...,4
2,... are not Duracell but for the price i am ha...,5
3,... as well as name brand batteries at a much ...,5
4,... batteries are very long lasting the price ...,5


,combined.review,reviews.rating
0,Too small I thought it would be as big as smal...,3
1,Great light reader. Easy to use at the beach T...,5
2,Great for the price Didnt know how much i'd us...,4
3,A Great Buy I am 100 happy with my purchase. I...,5
4,Solid entry-level Kindle. Great for kids Solid...,5


In [35]:
# Convert scores from [1,2,3] to -1, [4] to 0 and 5 to 1
train['score'] = train['reviews.rating'].apply(lambda rating: -1 if rating <= 3 else (0 if rating == 4 else 1))
test['score'] = test['reviews.rating'].apply(lambda rating: -1 if rating <= 3 else (0 if rating == 4 else 1))

display(train.info())
display(test.info())

# Convert the combined.review column to string
train['combined.review'] = train['combined.review'].astype(str)
test['combined.review'] = test['combined.review'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   combined.review  28332 non-null  object
 1   reviews.rating   28332 non-null  int64 
 2   score            28332 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 664.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   combined.review  4987 non-null   object
 1   reviews.rating   5000 non-null   int64 
 2   score            5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


None

In [36]:
'''# Lets inpect the training data

# number of datapoints with negative score -1
count_neg = train[train['score'] == -1].shape[0]
count_neu = train[train['score'] == 0].shape[0]
count_pos = train[train['score'] == 1].shape[0]
print(count_neg)
print(count_neu)
print(count_pos)

# our data has ~ 20k positive, 5,6k neutral and ~2.6k neg -> imbalanced data

#  we will delete the first 17110 positives
# Step 1: Find the indices of the first 17110 datapoints where score == 1
indices_to_remove = train[train['score'] == 1].index[:17110]

# Step 2: Drop those indices from the DataFrame
train_filtered = train.drop(indices_to_remove)

# we delete the first 2861 neutral
# Step 1: Find the indices of the first 2861 datapoints where score == 0
indices_to_remove_zero = train[train['score'] == 0].index[:2861]

# Step 2: Drop those indices from the DataFrame (including the previously removed indices)
train = train_filtered.drop(indices_to_remove_zero)


# Let's check again 
# number of datapoints with negative score -1
count_neg = train[train['score'] == -1].shape[0]
count_neu = train[train['score'] == 0].shape[0]
count_pos = train[train['score'] == 1].shape[0]
print(count_neg)
print(count_neu)
print(count_pos)
'''


"# Lets inpect the training data\n\n# number of datapoints with negative score -1\ncount_neg = train[train['score'] == -1].shape[0]\ncount_neu = train[train['score'] == 0].shape[0]\ncount_pos = train[train['score'] == 1].shape[0]\nprint(count_neg)\nprint(count_neu)\nprint(count_pos)\n\n# our data has ~ 20k positive, 5,6k neutral and ~2.6k neg -> imbalanced data\n\n#  we will delete the first 17110 positives\n# Step 1: Find the indices of the first 17110 datapoints where score == 1\nindices_to_remove = train[train['score'] == 1].index[:17110]\n\n# Step 2: Drop those indices from the DataFrame\ntrain_filtered = train.drop(indices_to_remove)\n\n# we delete the first 2861 neutral\n# Step 1: Find the indices of the first 2861 datapoints where score == 0\nindices_to_remove_zero = train[train['score'] == 0].index[:2861]\n\n# Step 2: Drop those indices from the DataFrame (including the previously removed indices)\ntrain = train_filtered.drop(indices_to_remove_zero)\n\n\n# Let's check again \n#

In [37]:
# Define a function to clean a sentence
def clean_sentence(sentence):
    import re

    # Remove any special character and replace them space " "
    cleaned_sentence = re.sub(r'[^a-zA-Z0-9\s]', " ", sentence)

    # Convert to lowercase
    cleaned_sentence = cleaned_sentence.lower()

    # Remove stopwords
    import nltk
    from nltk.corpus import stopwords
    ## Stopwords in english
    stopwords = stopwords.words("english")
    ## Remove stopwords from sentence
    for word in stopwords:
        cleaned_sentence = re.sub(r'\b' + re.escape(word) + r'\b', "", cleaned_sentence)
    
     # Replace multiple spaces with a single space
    cleaned_sentence = re.sub(r'\s+', ' ', cleaned_sentence)
    return cleaned_sentence

# Test the function
#test_function = clean_sentence("It's beyond my expectation, and it can even show music score. Not fast turning though.")
#print(test_function)

In [38]:
# Define a function to clean sentences from list
def clean_sentences(list_of_sentences):
    cleaned_sentences_list = []
    for sentence in list_of_sentences:
        cleaned_sentence = clean_sentence(sentence)
        cleaned_sentences_list.append(cleaned_sentence)
    return cleaned_sentences_list

# Test the function
#list_of_sentences_test = ["Hi, my name is Tiago and I'm a student at Ironhack","It's beyond my expectation, and it can even show music score. Not fast turning though."]
#test_function = clean_sentences(list_of_sentences_test)
#test_function


In [39]:
# Clean all text from train and test
train_text_clean = clean_sentences(train['combined.review'])
train['combined.review']= train_text_clean

test_text_clean = clean_sentences(test['combined.review'])
test['combined.review']= test_text_clean

# Inspect the final train and test data
display(train.head(10))
display(test.head(10))



,combined.review,reviews.rating,score
0,3 one item bad quality missing backup spring ...,3,-1
1,always less expensive way go products like bu...,4,0
2,duracell price happy well duracell price happy,5,1
3,well name brand batteries much better seem wo...,5,1
4,batteries long lasting price great batteries ...,5,1
5,batteries christmas amazonbasics cell good no...,5,1
6,batteries ordered past pleased ive problame b...,5,1
7,batteries last quite perfect nothing say well...,5,1
8,hold amount high power juice like energizer d...,3,-1
9,done well appear good shelf life amazonbasics...,4,0


,combined.review,reviews.rating,score
0,small thought would big small paper turn like...,3,-1
1,great light reader easy use beach kindle light...,5,1
2,great price didnt know much use kindle went lo...,4,0
3,great buy 100 happy purchase caught sale real...,5,1
4,solid entry level kindle great kids solid entr...,5,1
5,good ebook make excellent ebook reader expect ...,5,1
6,light weight makes world difference taking boo...,5,1
7,good quality bought kindle 2 months ago batte...,4,0
8,best ebook amazon kindle always best ebook upg...,5,1
9,great product beyond expectation even show mus...,5,1


In [40]:
train.shape

(28332, 3)

# Text vectorization

In [41]:
# Load a pre-trained embedding model (FastText) to vetorize our english sentences

import fasttext

# Load the pre-trained model
ft = fasttext.load_model('/Users/tiagovhp/Ironhack/Week_7/Amazon_SentimentAnalysis/cc.en.300.bin')



In [42]:

# define a funtion to vetorize a word
def word_to_vector(word):
    return ft.get_word_vector(word)

# Example: Get the vector for a specific word
test_function = word_to_vector("hello")
print(len(test_function))  # A 300-dimensional vector for the word 'hola'


# define a function to vetorize a sentence by avering the word vetors
def sentence_to_vector(sentence):

    list_of_vectors = []

    # tokenize the sentence
    words = sentence.lower().split()

    # get the vetor for each word and append to the list
    for word in words:
        vector = word_to_vector(word)
        list_of_vectors.append(vector)

    if len(list_of_vectors) == 0:
        # Return a zero vector of the same dimension as the word vectors
        return np.zeros(ft.get_dimension())
    
    # get a single vetor with the average of all vetors
    sentence_vector = np.mean(list_of_vectors,axis=0)
    return sentence_vector



300


Vectorize train data

In [43]:
# Vetorize our train text data
vectors_of_sentences = []
for text in train['combined.review']:
    sentence_vector = sentence_to_vector(text)
    vectors_of_sentences.append(sentence_vector)

# Convert list of sentence vectors into dataframe
vectorized_train = pd.DataFrame(vectors_of_sentences)

#train = train.join(vectors_of_sentences)
vectorized_train.head()



,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.016405,0.021548,0.025667,0.072047,-0.039506,0.017940,0.034434,0.007499,0.006417,0.001306,...,-0.010806,0.009702,-0.067697,-0.005992,-0.009765,0.008428,-0.010358,0.229547,-0.052881,-0.037996
1,0.006065,-0.005542,0.000476,0.092841,-0.096038,0.009971,0.073095,-0.036716,0.022067,-0.043828,...,0.068952,0.034941,-0.082531,-0.044198,0.017527,0.023170,0.015872,0.065651,-0.028809,-0.026347
2,0.013273,-0.004906,-0.042612,0.064305,-0.021589,0.011540,0.037645,-0.013416,0.001006,-0.015473,...,0.027085,-0.013314,-0.051359,0.024338,0.027586,0.006759,-0.023924,0.091058,-0.076720,-0.033665
3,-0.008748,0.034312,0.022642,0.065030,-0.040420,0.026625,0.055198,-0.014261,-0.011791,-0.003334,...,0.071402,-0.020617,-0.065556,-0.018781,-0.027825,-0.009369,-0.023736,0.090182,-0.056502,0.001286
4,0.006333,-0.022975,-0.042909,0.070586,-0.031547,0.020773,0.081795,-0.005218,-0.004390,-0.011015,...,0.035230,-0.003835,-0.023443,-0.019929,-0.008940,0.006224,-0.039597,0.079577,-0.045806,-0.045444


Vectorize test data

In [44]:
# Vetorize our test text data
vectors_of_sentences = []
for text in test['combined.review']:
    sentence_vector = sentence_to_vector(text)
    vectors_of_sentences.append(sentence_vector)

# Convert list of sentence vectors into dataframe
vectorized_test = pd.DataFrame(vectors_of_sentences)

#train = train.join(vectors_of_sentences)
vectorized_test.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.021915,-0.008435,0.003328,0.073585,-0.009047,0.006767,0.052753,0.000107,0.014670,-0.008109,...,0.035466,-0.004731,-0.069375,0.049152,-0.001504,0.011711,-0.007367,0.058627,0.005697,-0.013192
1,-0.001229,-0.008249,-0.019143,0.073841,0.036699,0.014730,0.098190,0.035266,0.034904,-0.019257,...,0.081358,-0.021478,-0.076344,-0.005835,-0.024213,0.001957,-0.022482,0.089050,-0.016593,-0.022671
2,-0.026833,-0.015601,-0.023259,0.080348,-0.024193,-0.007901,0.008676,0.008639,0.029717,-0.019626,...,0.074000,-0.014807,-0.105879,0.028833,0.005980,0.033689,-0.018360,0.075401,-0.036011,-0.050844
3,-0.004889,0.017907,0.012119,0.081755,-0.013358,0.004483,0.023578,-0.009126,-0.005628,-0.011432,...,0.020912,-0.000598,-0.070183,0.015242,-0.001649,0.015065,-0.009679,0.093725,-0.032341,-0.020512
4,0.000287,0.018261,-0.018091,0.073388,-0.004885,-0.011947,0.047967,-0.010661,-0.036683,-0.017548,...,0.044539,0.018230,-0.086413,0.005240,0.031126,0.026671,-0.033523,0.083704,-0.030887,-0.048870


# Split data into Train & Test

In [45]:
# Training

X_train = vectorized_train
y_train = train['score']

# Test
X_test = vectorized_test
y_test = test['score']



# Model selection

In [46]:
'''  
Here we make a preliminar analysis of traditional models. The aim is to compare the performance of each model (with default hyperparameters)
'''
# Import traditional models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

models = {"LogisticRegression" : LogisticRegression(class_weight='balanced'),
          "BernoulliNB" : BernoulliNB(),
          "SVC" : SVC(class_weight='balanced'),
          "RandomForestClassifier" : RandomForestClassifier(class_weight='balanced'),
          "DecisionTreeClassifier" : DecisionTreeClassifier(class_weight='balanced'),
          }
 

In [47]:
# lets perform cross validation and compare the performance
from sklearn.model_selection import cross_val_score

for name, model in models.items():
    try:
        scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
        print(name, " avg accuracy of: ", scores.mean())
    except ValueError as e:
        print(f"{name} encountered an error: {e}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression  avg accuracy of:  0.5477487620223249
BernoulliNB  avg accuracy of:  0.40099017036289447
SVC  avg accuracy of:  0.6393059289944801
RandomForestClassifier  avg accuracy of:  0.8757634115208397
DecisionTreeClassifier  avg accuracy of:  0.8133597818097243


# Hyperparameter tunning

In [48]:
# after cross validation between several models we select the RandomForestClassifier with a avg accuracy of:  0.88 (default param)
# Lets do hyperparameter tunning
param_grid = {'n_estimators': [350],
    'max_depth': [None],
    'min_samples_split': [10],
    'min_samples_leaf': [1],
    #'max_features': ['auto', 'sqrt', 'log2']
} 

In [49]:
# Grid search
# import library
from sklearn.model_selection import GridSearchCV

# Set up the Grid search
grid_search = GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced'),param_grid=param_grid,scoring = 'accuracy' , cv = 5, n_jobs=-1, verbose=1)

# fit the grid                 
grid_search.fit(X_train,y_train)

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy found: ", grid_search.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


Python(30498) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30499) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30500) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30501) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30502) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30503) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30504) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30505) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30506) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(30507) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 350}
Best accuracy found:  0.8773868703514524


# Train the Model

In [50]:
# We now train the best model with best hyperparametera

# Instantiate the model with best hyperparameters
trad_model = RandomForestClassifier(class_weight='balanced',max_depth= None, min_samples_leaf = 1, min_samples_split = 10, n_estimators = 250)

# Train the model
trad_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', min_samples_split=10,
                       n_estimators=250)

# Model Test 

In [51]:
# Make predctions / test
y_pred = trad_model.predict(X_test)



# Performance metrics

In [52]:
# Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Calculate evaluation metrics using weighted averaging
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalance
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision (weighted): {precision:.2f}")
print(f"Recall (weighted): {recall:.2f}")
print(f"F1-score (weighted): {f1:.2f}")

Accuracy: 0.92
Precision (weighted): 0.93
Recall (weighted): 0.92
F1-score (weighted): 0.92


In [53]:
# Calculate and display confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
class_labels = ["Class -1", "Class 0", "Class 1"]
conf_matrix_df = pd.DataFrame(conf_matrix, index=class_labels, columns=class_labels)
print("\nConfusion Matrix:")
print(conf_matrix_df)


Confusion Matrix:
          Class -1  Class 0  Class 1
Class -1       242        4       68
Class 0          0      902      306
Class 1          0        0     3478
